In [1]:
import datetime
import requests
import boto3
import pandas as pd
import io
import re

In [2]:
url_template = 'https://www.twse.com.tw/holidaySchedule/holidaySchedule?response=csv&queryYear={}'
filename_tamplate = 'holidaySchedule_{}.csv'

def get_tw_year(year: int) -> int:
    return year - 1911

year = datetime.datetime.today().year
tw_year = get_tw_year(year)

url = url_template.format(tw_year)
filename = filename_tamplate.format(tw_year)

s3_bucket = 'indextracker'
s3_key = f'tw/{filename}'

## load data directly

In [3]:
def download_file(filepath, url):
    with requests.get(url, stream=True) as response:
        with open(filepath, 'wb') as f:
            for content in response.iter_content():
                f.write(content)

# download_file(f'.\{filename}', url)

In [4]:
def check_s3_key_exist(bucket, key):
    from botocore.exceptions import ClientError
    
    try:
        s3c = boto3.client('s3')
        s3c.head_object(Bucket=bucket, Key=key)
        return True
    
    except ClientError: # https://stackoverflow.com/a/42978638
        return False

# check_s3_key_exist(s3_bucket, s3_key)

In [5]:
def download_to_s3(bucket, key, url):
    with requests.get(url, stream=True) as response:
        s3c = boto3.client('s3')
        s3c.upload_fileobj(response.raw, bucket, key)
    
# download_to_s3(s3_bucket, s3_key, url)

In [6]:
content = requests.get(url).content.decode('big5').split('\n')
content

['中華民國111年有價證券集中交易市場開（休）市日期表',
 '名稱,日期,星期,說明,備註(* : 市場無交易，僅辦理結算交割作業。o : 交易日。)',
 '"中華民國開國紀念日","1月1日","六","依規定放假1日。",""',
 '"國曆新年開始交易日","1月3日","一","國曆新年開始交易。","o"',
 '"農曆春節前最後交易日","1月26日","三","農曆春節前最後交易。","o"',
 '"農曆春節前最後交易日","1月27日","四","1月27日市場無交易，僅辦理結算交割作業。","*"',
 '"農曆春節前最後交易日","1月28日","五","1月28日市場無交易，僅辦理結算交割作業。","*"',
 '"農曆除夕","1月31日","一","依規定放假1日。",""',
 '"農曆春節","2月1日","二","依規定放假1日。",""',
 '"農曆春節","2月2日","三","依規定放假1日。",""',
 '"農曆春節","2月3日","四","依規定放假1日。",""',
 '"農曆春節","2月4日","五","2月4日（星期五）調整放假，於1月22日（星期六）補行上班，但不交易亦不交割。",""',
 '"農曆春節後開始交易日","2月7日","一","農曆春節後開始交易。","o"',
 '"和平紀念日","2月28日","一","依規定放假1日。",""',
 '"兒童節","4月4日","一","依規定放假1日。",""',
 '"民族掃墓節","4月5日","二","依規定放假1日。",""',
 '"勞動節","5月1日","日","依規定放假1日。",""',
 '"勞動節","5月2日","一","5月1日適逢星期日，5月2日（星期一）補假1日。",""',
 '"端午節","6月3日","五","依規定放假1日。",""',
 '"中秋節","9月9日","五","9月10日適逢星期六，9月9日（星期五）補假1日。",""',
 '"中秋節","9月10日","六","依規定放假1日。",""',
 '"國慶日","10月10日","一","依規定放假1日。",""',
 '']

In [7]:
df:pd.DataFrame = pd.read_csv(io.StringIO('\n'.join(content[1:])), encoding='big5')
df

,名稱,日期,星期,說明,備註(* : 市場無交易，僅辦理結算交割作業。o : 交易日。)
0,中華民國開國紀念日,1月1日,六,依規定放假1日。,NaN
1,國曆新年開始交易日,1月3日,一,國曆新年開始交易。,o
2,農曆春節前最後交易日,1月26日,三,農曆春節前最後交易。,o
3,農曆春節前最後交易日,1月27日,四,1月27日市場無交易，僅辦理結算交割作業。,*
4,農曆春節前最後交易日,1月28日,五,1月28日市場無交易，僅辦理結算交割作業。,*
5,農曆除夕,1月31日,一,依規定放假1日。,NaN
6,農曆春節,2月1日,二,依規定放假1日。,NaN
7,農曆春節,2月2日,三,依規定放假1日。,NaN
8,農曆春節,2月3日,四,依規定放假1日。,NaN
9,農曆春節,2月4日,五,2月4日（星期五）調整放假，於1月22日（星期六）補行上班，但不交易亦不交割。,NaN


In [8]:
df.columns = ['name', 'date', 'weekday', 'description', 'memo']
df

,name,date,weekday,description,memo
0,中華民國開國紀念日,1月1日,六,依規定放假1日。,NaN
1,國曆新年開始交易日,1月3日,一,國曆新年開始交易。,o
2,農曆春節前最後交易日,1月26日,三,農曆春節前最後交易。,o
3,農曆春節前最後交易日,1月27日,四,1月27日市場無交易，僅辦理結算交割作業。,*
4,農曆春節前最後交易日,1月28日,五,1月28日市場無交易，僅辦理結算交割作業。,*
5,農曆除夕,1月31日,一,依規定放假1日。,NaN
6,農曆春節,2月1日,二,依規定放假1日。,NaN
7,農曆春節,2月2日,三,依規定放假1日。,NaN
8,農曆春節,2月3日,四,依規定放假1日。,NaN
9,農曆春節,2月4日,五,2月4日（星期五）調整放假，於1月22日（星期六）補行上班，但不交易亦不交割。,NaN


In [9]:
def extract_date(date_string, year) -> datetime.date:
    m, d = re.search('(\d+)月(\d+)日', date_string).groups()
    return datetime.date(year, int(m), int(d))
    
df['ds'] = df['date'].apply(extract_date, year=year)
df['is_workday'] = df['memo'].apply(lambda x: True if x=='o' else False)

df

,name,date,weekday,description,memo,ds,is_workday
0,中華民國開國紀念日,1月1日,六,依規定放假1日。,NaN,2022-01-01,False
1,國曆新年開始交易日,1月3日,一,國曆新年開始交易。,o,2022-01-03,True
2,農曆春節前最後交易日,1月26日,三,農曆春節前最後交易。,o,2022-01-26,True
3,農曆春節前最後交易日,1月27日,四,1月27日市場無交易，僅辦理結算交割作業。,*,2022-01-27,False
4,農曆春節前最後交易日,1月28日,五,1月28日市場無交易，僅辦理結算交割作業。,*,2022-01-28,False
5,農曆除夕,1月31日,一,依規定放假1日。,NaN,2022-01-31,False
6,農曆春節,2月1日,二,依規定放假1日。,NaN,2022-02-01,False
7,農曆春節,2月2日,三,依規定放假1日。,NaN,2022-02-02,False
8,農曆春節,2月3日,四,依規定放假1日。,NaN,2022-02-03,False
9,農曆春節,2月4日,五,2月4日（星期五）調整放假，於1月22日（星期六）補行上班，但不交易亦不交割。,NaN,2022-02-04,False


In [10]:
makeup_days = pd.to_datetime(df[df['is_workday']].ds)
holidays = pd.to_datetime(df[~df['is_workday']].ds)

def is_trading_days(date: datetime.date) -> bool:
    is_weekend = date.weekday() >=  5
    is_holiday = (holidays == date).any()
    is_makeup_day = (makeup_days == date).any()
    
    return (not is_weekend and not is_holiday) or is_makeup_day

In [18]:
first_day_of_year = datetime.date(year, 1, 1)
last_day_of_year = datetime.date(year, 12, 31)
# all_days = pd.date_range(first_day_of_year, last_day_of_year).tolist()[0].dayofweek # 0:mon ~ 6:sun
all_days:pd.Series = pd.date_range(first_day_of_year, last_day_of_year).to_series()

all_days[~all_days.apply(is_trading_days)]

2022-01-01   2022-01-01
2022-01-02   2022-01-02
2022-01-08   2022-01-08
2022-01-09   2022-01-09
2022-01-15   2022-01-15
                ...    
2022-12-17   2022-12-17
2022-12-18   2022-12-18
2022-12-24   2022-12-24
2022-12-25   2022-12-25
2022-12-31   2022-12-31
Length: 119, dtype: datetime64[ns]